In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [2]:
print('Example of layered model with a function task (FaaS)')

model = LayeredNetwork('faas_test_example')

# definition of processors, tasks and entries
P1 = Processor(model, 'P1', GlobalConstants.MaxInt, SchedStrategy.INF)
T1 = Task(model, 'T1', 1, SchedStrategy.REF).on(P1)
E1 = Entry(model, 'E1').on(T1)

P2 = Processor(model, 'P2', 4, SchedStrategy.FCFS)
# Function task with capacity, scheduling strategy, think time, setup time, and delay-off time
T2 = FunctionTask(model, 'F2', 6, SchedStrategy.FCFS).on(P2).setThinkTime(Exp.fitMean(8.0))
T2.setSetupTime(Exp(1.0))      # Cold start time
T2.setDelayOffTime(Exp(2.0))   # Time before function instance is removed

E2 = Entry(model, 'E2').on(T2)

# definition of activities
A1 = Activity(model, 'A1', Exp(1.0)).on(T1).boundTo(E1).synchCall(E2, 1)
A2 = Activity(model, 'A2', Exp(3.0)).on(T2).boundTo(E2).repliesTo(E2)

Example of layered model with a function task (FaaS)


In [3]:
# Define a helper function for solver selection (as in original MATLAB)
def myFunction(model):
    """Helper function to select appropriate solver for the model"""
    # This could implement logic to choose between MVA, MAM, etc.
    # For now, use MVA as default
    options = SolverMVA.defaultOptions()
    options.verbose = 0
    return SolverMVA(model, options)

# Solve with LN solver using custom function
lnoptions = SolverLN.defaultOptions()
lnoptions.verbose = 0
lnoptions.seed = 2300

solver = SolverLN(model, myFunction, lnoptions)
avg_table = solver.get_avg_table()

SolverLN initialization completed. Starting iteration on ensemble models.
Iter 1.Analyze time: 0.021339s. Update time: 0.001145s. Runtime: 0.021647s.
Iter 2.Analyze time: 0.001382s. Update time: 0.000173s. Runtime: 0.024709s.
Iter 3.Analyze time: 0.001268s. Update time: 0.000230s. Runtime: 0.026385sTesting convergence.

Iter 4.Analyze time: 0.004079s. Update time: 0.000214s. Runtime: 0.030923s.
SolverLN completed in 3 iterations. Running final checks.\n
Summary: Analyze avg time: 0.007017s. Update avg time: 0.000440s. Total runtime: 0.032257s. 
  Node   NodeType  QLen  Util  RespT  ResidT  ArvR   Tput
0   P1  Processor   NaN   1.0    NaN     NaN   NaN    NaN
1   P2  Processor   NaN   0.0    NaN     NaN   NaN    NaN
2   T1       Task   1.0   1.0    NaN     1.0   NaN  1.000
3   F2       Task   0.0   0.0    NaN     0.0   NaN  0.125
4   E1      Entry   1.0   NaN    1.0     NaN   NaN  1.000
5   E2      Entry   0.0   NaN    0.0     NaN   NaN  0.125
6   A1   Activity   1.0   1.0    1.0     1.